In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read Excel file
df = pd.read_excel(r"C:\Users\HP\Project\MLL_1.xlsx")

In [3]:
df

,class,31307,31308_at,31309_r_at,31310_at,31311_at,31312_at,31313_at,31314_at,31315_at,...,100_g_at,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at
0,ALL,-135.7,-100.1,-94.6,-230,0.6,-50.4,-36.3,139.5,31.6,...,548.7,-225.2,242.5,101.7,473.1,-59.9,217.9,275.6,-461.6,1115.5
1,ALL,-80.0,-23.0,-6.0,-145,491.0,290.0,-235.0,41.0,4602.0,...,1464.0,-175.0,143.0,96.0,301.0,-50.0,242.0,222.0,-330.0,2481.0
2,ALL,-91.0,-130.0,-27.0,-51,236.0,-163.0,-304.0,-35.0,498.0,...,339.0,-308.0,184.0,-32.0,350.0,-11.0,837.0,174.0,-99.0,376.0
3,ALL,-144.0,-124.0,-26.0,-139,-88.0,34.0,-411.0,118.0,-239.0,...,798.0,731.0,106.0,-330.0,-36.0,-190.0,999.0,255.0,-353.0,1603.0
4,ALL,-89.0,-25.0,-64.0,-112,452.0,183.0,107.0,233.0,38.0,...,730.0,182.0,426.0,155.0,607.0,50.0,249.0,1635.0,-780.0,1103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,AML,-324.0,-168.0,-49.0,312,1059.0,-24.0,-404.0,12.0,101.0,...,1808.0,-422.0,528.0,220.0,643.0,187.0,407.0,-564.0,-1736.0,346.0
68,AML,-148.0,-104.0,29.0,72,465.0,162.0,-895.0,33.0,1736.0,...,2913.0,128.0,94.0,66.0,556.0,63.0,200.0,120.0,-757.0,825.0
69,AML,-230.0,-66.0,-69.0,377,686.0,-44.0,-123.0,7.0,310.0,...,2555.0,-230.0,257.0,71.0,581.0,64.0,35.0,829.0,-2015.0,385.0
70,AML,-359.0,-52.0,-147.0,120,564.0,-52.0,-584.0,64.0,2528.0,...,1982.0,-236.0,88.0,94.0,143.0,232.0,434.0,-87.0,-2038.0,1228.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 12534 entries, class to 109_at
dtypes: float64(11224), int64(1309), object(1)
memory usage: 6.9+ MB


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [8]:
# -----------------------------
# 1️ Load & clean dataset
# -----------------------------
df = pd.read_excel(r"C:\Users\HP\Project\MLL_1.xlsx", sheet_name='Sheet1')
df_clean = df.iloc[2:].reset_index(drop=True)  # remove first 2 rows

# Ensure all column names are strings to avoid KeyError later
df_clean.columns = df_clean.columns.astype(str)

# -----------------------------
# 2️ Separate features and target
# -----------------------------
X = df_clean.drop(columns=['class']).astype(float)
X.columns = X.columns.astype(str)  # ensure column names are strings
y = df_clean['class']

# -----------------------------
# 3️ Scale all features for autoencoder
# -----------------------------
scaler_all = StandardScaler()
X_scaled_all = scaler_all.fit_transform(X)

# -----------------------------
# 4️ Folded Autoencoder for feature selection
# -----------------------------
hidden_dim = 5
autoencoder = MLPRegressor(hidden_layer_sizes=(hidden_dim,),
                           max_iter=100,
                           random_state=0)
autoencoder.fit(X_scaled_all, X_scaled_all)

# -----------------------------
# 5️ Compute feature importance
# -----------------------------
coefs = autoencoder.coefs_[0]  # weights from input layer to hidden layer
feature_importance = np.sum(np.abs(coefs), axis=1)

# -----------------------------
# 6️ Select top 20 features
# -----------------------------
k = 20
top_features_idx = np.argsort(feature_importance)[::-1][:k]
top_features = X.columns[top_features_idx]

print("Top 20 features:")
print(top_features)

# -----------------------------
# 7️ Shuffle & normalize only top 20 features
# -----------------------------
df_selected = df_clean[top_features.tolist() + ['class']]
df_shuffled = df_selected.sample(frac=1, random_state=42).reset_index(drop=True)

X_top = df_shuffled[top_features].astype(float)
y_top = df_shuffled['class']

scaler_top = StandardScaler()
X_top_scaled = scaler_top.fit_transform(X_top)

# -----------------------------
# 8️ Optional: combine scaled features with target
# -----------------------------
df_final = pd.DataFrame(X_top_scaled, columns=top_features)
df_final['class'] = y_top.values

# -----------------------------
# Now ready for modeling
# X_top_scaled -> normalized top 20 features
# y_top -> target labels
# df_final -> shuffled + normalized dataframe

Top 20 features:
Index(['41589_at', '34959_at', '40849_s_at', '34859_at', '32685_at',
       '33489_at', '32293_at', '33439_at', '35621_at', '36669_at', '32254_at',
       '34605_at', '35855_s_at', '39651_at', '33180_at', '39706_at',
       '37289_at', '33298_at', '37805_at', '38107_at'],
      dtype='object')


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
df_final

,41589_at,34959_at,40849_s_at,34859_at,32685_at,33489_at,32293_at,33439_at,35621_at,36669_at,...,34605_at,35855_s_at,39651_at,33180_at,39706_at,37289_at,33298_at,37805_at,38107_at,class
0,0.638247,0.081252,-0.332653,-0.354327,-0.722262,0.306102,0.173950,0.641421,0.135596,-0.047622,...,-1.084377,0.150879,-0.469394,-0.794542,-0.305796,0.696188,0.122672,1.202397,-0.289912,MLL
1,-1.438738,-0.044519,-0.981880,-1.399806,-0.492985,-0.369252,-0.046029,-0.116986,0.003367,-0.009619,...,0.553105,-0.220533,-0.359194,-0.376628,-0.789986,0.569996,-0.145673,1.331436,-0.952776,ALL
2,0.216958,0.032341,-1.571125,-0.721441,0.340241,-1.370002,0.018670,-0.653136,1.348651,-0.492371,...,-0.441514,0.100157,-0.351180,-0.143560,0.379470,0.707660,0.149507,0.049649,-1.202790,AML
3,0.599948,-0.236669,-0.590227,1.539773,2.431694,-0.811301,-0.078379,-0.721064,0.109725,-1.079349,...,0.589494,-0.254893,0.021496,-0.776861,0.911919,0.179947,0.142798,1.030345,0.047277,ALL
4,-0.224954,-0.128366,0.762918,1.245815,-1.180816,-0.442926,0.335699,1.616412,0.696131,2.009623,...,-0.720492,0.145970,-0.868117,0.097544,-0.622691,-0.313350,0.686197,0.772267,-0.444526,AML
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.464428,1.960833,-1.059505,0.333349,-0.146274,-0.418368,0.025140,2.589366,-0.094367,2.154298,...,0.674400,0.296498,0.181787,0.333826,-0.485960,0.535580,0.129381,0.428164,0.585135,MLL
66,0.228742,0.220998,0.216014,0.030080,0.983335,-1.093721,0.031610,-0.762374,1.549869,-0.811446,...,0.019407,0.975510,0.802913,-0.961708,0.046489,-0.255990,0.632528,0.350740,0.154191,ALL
67,-0.145410,0.538920,-1.902795,0.393204,-0.789368,-0.295577,-0.278948,-0.337206,-1.189566,-0.130777,...,1.438558,1.345286,-1.795799,1.873680,-0.162630,1.051821,0.437978,-0.070787,1.364781,AML
68,1.177379,-0.631451,-0.170347,-0.024455,0.155701,-1.056884,0.154540,0.788768,0.244829,-0.362558,...,-0.405125,0.685907,0.187798,2.204797,-0.175499,-0.278934,-1.185511,0.032444,0.249590,ALL


In [15]:
# Display distinct values of a column, e.g. 'City'
unique_values = df['class'].unique()

print(unique_values)

['ALL' 'MLL' 'AML']


In [16]:
# Count empty (NaN) cells in each column
empty_counts = df.isna().sum()

# Print the result
print("Number of empty cells in each column:")
print(empty_counts)

Number of empty cells in each column:
class         0
31307         0
31308_at      0
31309_r_at    0
31310_at      0
             ..
105_at        0
106_at        0
107_at        0
108_g_at      0
109_at        0
Length: 12534, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# -----------------------------
# 1️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_top_scaled, y_top, test_size=0.2, random_state=42, stratify=y_top
)

# -----------------------------
# 2️⃣ Define Random Forest model
# -----------------------------
rf_model = RandomForestClassifier(
    n_estimators=100,       # number of trees
    max_depth=None,         # grow trees fully
    random_state=42
)

# -----------------------------
# 3️⃣ Train the model
# -----------------------------
rf_model.fit(X_train, y_train)

# -----------------------------
# 4️⃣ Make predictions
# -----------------------------
y_pred = rf_model.predict(X_test)

# -----------------------------
# 5️⃣ Evaluate the model
# -----------------------------
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# -----------------------------
# Confusion Matrix
# -----------------------------
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# -----------------------------
# Precision and Recall (per class)
# -----------------------------
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)

print("\nPrecision per class:", precision)
print("Recall (Sensitivity) per class:", recall)

# -----------------------------
# Specificity and Negative Predictive Value (NPV)
# -----------------------------
specificity = []
npv = []
n_classes = len(np.unique(y_test))

for i in range(n_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    spec = TN / (TN + FP) if (TN + FP) != 0 else 0
    npv_val = TN / (TN + FN) if (TN + FN) != 0 else 0
    
    specificity.append(spec)
    npv.append(npv_val)

print("Specificity per class:", specificity)
print("Negative Predictive Value (NPV) per class:", npv)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# -----------------------------
# 1️⃣ Define RandomForest and parameter grid
# -----------------------------
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
}

# -----------------------------
# 2️⃣ Grid Search with 5-fold cross-validation
# -----------------------------
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,  # use all cores
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# -----------------------------
# 3️⃣ Evaluate on test set
# -----------------------------
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

test_acc = accuracy_score(y_test, y_pred_best)
print("\nTest Accuracy with Best RF:", test_acc)
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_best))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))